<a href="https://colab.research.google.com/github/daradanci/MMO_2025/blob/main/notes/LR6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №6  
## Классификация текстов

**Цель работы:** изучить методы классификации текстов на основе различных способов векторизации.  
**Задание:** Реализовать два подхода к классификации текстов:  
1. На основе `TfidfVectorizer`.  
2. На основе `word2vec`, обученного вручную на корпусе.  
Сравнить полученные результаты классификации.


### Используемый набор данных

В качестве источника данных выбран встроенный корпус `20 Newsgroups` из библиотеки `sklearn.datasets`.  
Из 20 доступных категорий были выбраны две для бинарной классификации:

- `sci.space` — научные обсуждения, связанные с космосом;
- `rec.sport.baseball` — обсуждения бейсбольных матчей и команд.

Это позволяет продемонстрировать алгоритмы классификации на реальных текстах, отличающихся тематически.


In [2]:
!pip install numpy==1.24.3
!pip install gensim


In [3]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# Выбираем 2 категории для бинарной классификации
categories = ['sci.space', 'rec.sport.baseball']

# Загружаем данные
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

# Формируем DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})

# Просмотр примера
df.head(3)


________________________________________________________________________________
Cache loading failed
________________________________________________________________________________
No module named 'numpy._core'


,text,label
0,"I've been saying this for quite some time, but...",0
1,Sorry for asking a question that's not entirel...,1
2,Giant's have a five man rotation of John Burk...,0


### Способ 1. Классификация на основе TfidfVectorizer

Для первого подхода тексты были преобразованы в числовые векторы с помощью `TfidfVectorizer`, исключающего стоп-слова и учитывающего частотность слов.  
Классификация выполнялась с помощью модели `LogisticRegression`, которая хорошо работает на задачах с разреженными признаками.  
Ниже приведён отчёт о качестве модели на тестовой выборке.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Векторизация текста
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение логистической регрессии
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Оценка модели
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred, target_names=newsgroups.target_names))


                    precision    recall  f1-score   support

rec.sport.baseball       0.95      0.90      0.92       130
         sci.space       0.89      0.94      0.91       108

          accuracy                           0.92       238
         macro avg       0.92      0.92      0.92       238
      weighted avg       0.92      0.92      0.92       238



### Способ 2. Классификация на основе Word2Vec

Во втором подходе используется модель `Word2Vec`, обученная на тексте из датасета `20 Newsgroups`.  
Каждое слово кодируется вектором, отражающим его контекст, а вектор документа получаем как среднее всех слововых векторов.  
Обучение выполняется с использованием библиотеки `gensim`.


In [9]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # иногда помогает отдельно
nltk.download('popular')    # качает всё базовое, включая punkt и токенизаторы


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   

True

In [10]:
# import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
# import nltk
import numpy as np

nltk.download('punkt')

# Токенизируем все тексты для обучения модели
tokenized_texts = [word_tokenize(text.lower()) for text in df['text']]

# Обучаем Word2Vec на корпусе
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=2, workers=4, sg=1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Получение признаков документов на основе Word2Vec

Каждый документ преобразуется в вектор путём усреднения эмбеддингов слов, присутствующих в нём.  
Если все слова документа отсутствуют в словаре модели, возвращается нулевой вектор.  
Таким образом, создаются числовые представления текстов, пригодные для подачи в классификатор.


In [12]:
from nltk.tokenize import word_tokenize

# Функция усреднения эмбеддингов слов в документе
def document_vector(doc, model):
    words = [word for word in word_tokenize(doc.lower()) if word in model.wv.key_to_index]
    if not words:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

# Преобразуем все документы в векторы
X_w2v = np.array([document_vector(text, w2v_model) for text in df['text']])
y = df['label']

# Делим выборку
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size=0.2, random_state=42)


### Классификация на Word2Vec-векторах

После получения векторного представления документов, была обучена модель `LogisticRegression`.  
Модель показала определённый уровень качества классификации, позволяющий сравнить её с результатами, полученными при использовании `TfidfVectorizer`.


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Обучаем модель на векторах Word2Vec
model_w2v = LogisticRegression(max_iter=1000)
model_w2v.fit(X_train_w2v, y_train_w2v)

# Предсказания
y_pred_w2v = model_w2v.predict(X_test_w2v)

# Выводим отчёт
print(classification_report(y_test_w2v, y_pred_w2v, target_names=newsgroups.target_names))


                    precision    recall  f1-score   support

rec.sport.baseball       0.88      0.88      0.88       130
         sci.space       0.86      0.86      0.86       108

          accuracy                           0.87       238
         macro avg       0.87      0.87      0.87       238
      weighted avg       0.87      0.87      0.87       238



---

### Сравнение моделей и вывод

В данной лабораторной работе были реализованы два подхода к классификации текстов:

1. **TfidfVectorizer + LogisticRegression**  
   Модель показала высокую точность благодаря учёту частоты слов в тексте и их значимости в корпусе.  
   Преимущество — простота и эффективность на небольших и средних наборах данных.

2. **Word2Vec + LogisticRegression**  
   Этот подход использует контекстные представления слов, усреднённые по каждому документу.  
   Хотя результат может быть ниже, модель способна улавливать семантику слов и подходит для более глубокого анализа текста.

📊 **Итог:**  
Обе модели успешно решили задачу классификации. Метод на основе `TfidfVectorizer` показал более высокую точность на выбранном корпусе, однако `Word2Vec` даёт более гибкое представление текста и может быть полезен при работе с более сложными задачами или большими данными.

Таким образом, цель лабораторной работы достигнута, оба метода реализованы и протестированы.
